In [38]:
import pandas as pd
import pysam
from xopen import xopen

In [55]:
df = pd.read_csv('/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20240227_1/A0204_7_MM_WXvazymeTN5_7th_D02SDS_T7_EDTA5030/outs/cell_qc_metrics.tsv',sep='\t')

In [56]:
df = df[df['cell_called']==True]

In [57]:
df = df.sort_values('fragments', ascending=False)

In [58]:
df = df.head(100)

In [59]:
target_bc = set(df.barcode)

In [60]:
index_res = set()

In [61]:
index = 0
with pysam.FastxFile('/SGRNJ07/Standard_Analysis/DATA05/limsdownload/24_02/2024_02_22/RD22110301/A0204_7_MM_WXvazymeTN5_7th_D02SDS_T7_EDTA5030/2024-02-22-14/R240206043_R2.fastq.gz') as f:
    for read in f:
        index+=1
        bc = read.sequence[:6] + read.sequence[10:18] + read.sequence[22:28]
        if bc in target_bc:
            index_res.add(index)

In [62]:
fh_fq1 = xopen("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20240412testdata/rawdata/test_R1.fastq.gz", 'w')
fh_fq2 = xopen("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20240412testdata/rawdata/test_R2.fastq.gz", 'w')
fh_fq3 = xopen("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20240412testdata/rawdata/test_R3.fastq.gz", 'w')
fh_index1 = xopen("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20240412testdata/rawdata/test_I1.fastq.gz", 'w')

In [63]:
fq1 = "/SGRNJ07/Standard_Analysis/DATA05/limsdownload/24_02/2024_02_22/RD22110301/A0204_7_MM_WXvazymeTN5_7th_D02SDS_T7_EDTA5030/2024-02-22-14/R240206043_R1.fastq.gz"
fq2 = "/SGRNJ07/Standard_Analysis/DATA05/limsdownload/24_02/2024_02_22/RD22110301/A0204_7_MM_WXvazymeTN5_7th_D02SDS_T7_EDTA5030/2024-02-22-14/R240206043_R2.fastq.gz"
fq3 = "/SGRNJ07/Standard_Analysis/DATA05/limsdownload/24_02/2024_02_22/RD22110301/A0204_7_MM_WXvazymeTN5_7th_D02SDS_T7_EDTA5030/2024-02-22-14/R240206043_R3.fastq.gz"
index_fq = "/SGRNJ07/Standard_Analysis/DATA05/limsdownload/24_02/2024_02_22/RD22110301/A0204_7_MM_WXvazymeTN5_7th_D02SDS_T7_EDTA5030/2024-02-22-14/R240206043_I1.fastq.gz"

In [64]:
count = 0
with pysam.FastxFile(fq1, persist=False) as fq1, \
        pysam.FastxFile(fq2, persist=False) as fq2, \
            pysam.FastxFile(fq3, persist=False) as fq3, \
                pysam.FastxFile(index_fq, persist=False) as index1:
    for entry1, entry2, entry3, entry4 in zip(fq1, fq2, fq3, index1):
        count += 1
        if count in index_res:
            fh_fq1.write(f"@{entry1.name}\n{entry1.sequence}\n+\n{entry1.quality}\n")
            fh_fq2.write(f"@{entry2.name}\n{entry2.sequence}\n+\n{entry2.quality}\n")
            fh_fq3.write(f"@{entry3.name}\n{entry3.sequence}\n+\n{entry3.quality}\n")
            fh_index1.write(f"@{entry4.name}\n{entry4.sequence}\n+\n{entry4.quality}\n")
fh_fq1.close()
fh_fq2.close()
fh_fq3.close()
fh_index1.close()